# importing

In [21]:
import os
import pandas as pd
import numpy as np
import cv2
import glob
import shutil
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.data import AUTOTUNE



# Loading 

In [24]:
import pandas as pd

# Load CSV files from the "csv" directory
calc_train = pd.read_csv("csv/calc_case_description_train_set.csv")
calc_test = pd.read_csv("csv/calc_case_description_test_set.csv")
mass_train = pd.read_csv("csv/mass_case_description_train_set.csv")
mass_test = pd.read_csv("csv/mass_case_description_test_set.csv")
dicom_info = pd.read_csv("csv/dicom_info.csv")
meta = pd.read_csv("csv/meta.csv")

# Store them in a dictionary for easier access
csv_files = {
    "calc_train": calc_train,
    "calc_test": calc_test,
    "mass_train": mass_train,
    "mass_test": mass_test,
    "dicom_info": dicom_info,
    "meta": meta
}


In [26]:
import tensorflow as tf
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))


GPUs Available:  []


In [28]:
train_df=pd.read_csv('train_df.csv')
test_df=pd.read_csv('test_df.csv')

In [30]:
train_df.head()

,image_path,label
0,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,1
1,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,1
2,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,0
3,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,0
4,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,0


In [32]:
test_df.head()

,image_path,label
0,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,1
1,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,1
2,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,1
3,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,1
4,E:/Breast Cancer/jpeg\1.3.6.1.4.1.9590.100.1.2...,0


# preprocessing

# TensorFLow

In [36]:
# Image Parameters
IMG_SIZE = 380  # EfficientNet-B4 expects 380x380 images
BATCH_SIZE = 32

def load_and_preprocess_image(path, label):
    """Load an image and preprocess it for EfficientNetB4."""
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # Decode JPEG
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])  # Resize to 380x380
    image = tf.keras.applications.efficientnet.preprocess_input(image)  # Normalize for EfficientNet
    return image, label

def dataframe_to_tf_dataset(df, shuffle=True):
    """Convert a Pandas DataFrame to a TensorFlow dataset."""
    file_paths = df['image_path'].values
    labels = df['label'].values
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(len(df))  # Shuffle dataset
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)  # Optimize performance
    return dataset


In [38]:

# Convert to TensorFlow dataset
train_dataset = dataframe_to_tf_dataset(train_df)
test_dataset = dataframe_to_tf_dataset(test_df, shuffle=False)  # No need to shuffle test set


# Load Pretrained EfficientNet-B4 Model

In [21]:
# Load pre-trained EfficientNet-B4 (without top layers)
base_model = EfficientNetB4(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model
base_model.trainable = False

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)  # Prevent overfitting
output_layer = Dense(1, activation="sigmoid")(x)  # Binary classification (0 or 1)

# Create final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=["accuracy"])


# Train the Model

In [31]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath="breast_cancer_model.h5",  # File path to save the model
    save_weights_only=False,  # Save the full model (architecture + weights)
    save_best_only=False,  # Save the model after every epoch, not just the best one
    verbose=1  # Print a message when saving
)

# Train the model with the checkpoint callback
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset,
    callbacks=[checkpoint_callback]  # Add checkpoint callback
)


Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6193 - loss: 0.6503
Epoch 1: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 279s 3s/step - accuracy: 0.6196 - loss: 0.6501 - val_accuracy: 0.6080 - val_loss: 0.6351
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6538 - loss: 0.6139
Epoch 2: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 278s 3s/step - accuracy: 0.6539 - loss: 0.6139 - val_accuracy: 0.6179 - val_loss: 0.6288
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6786 - loss: 0.5926
Epoch 3: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 276s 3s/step - accuracy: 0.6786 - loss: 0.5926 - val_accuracy: 0.6335 - val_loss: 0.6159
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6827 - loss: 0.5904
Epoch 4: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 272s 3s/step - accuracy: 0.6828 - loss: 0.5903 - val_accuracy: 0.6335 - val_loss: 0.6106
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7182 - loss: 0.5604
Epoch 5: saving model to breast_cancer_model.h5


...

90/90 ━━━━━━━━━━━━━━━━━━━━ 269s 3s/step - accuracy: 0.7144 - loss: 0.5543 - val_accuracy: 0.6392 - val_loss: 0.6086
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7307 - loss: 0.5391
Epoch 7: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 268s 3s/step - accuracy: 0.7306 - loss: 0.5392 - val_accuracy: 0.6534 - val_loss: 0.6034
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7300 - loss: 0.5311
Epoch 8: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 269s 3s/step - accuracy: 0.7300 - loss: 0.5311 - val_accuracy: 0.6605 - val_loss: 0.5952
Epoch 9/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7363 - loss: 0.5288
Epoch 9: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 280s 3s/step - accuracy: 0.7363 - loss: 0.5288 - val_accuracy: 0.6562 - val_loss: 0.5900
Epoch 10/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7257 - loss: 0.5307
Epoch 10: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 268s 3s/step - accuracy: 0.7258 - loss: 0.5306 - val_accuracy: 0.6406 - val_loss: 0.6017


In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath="breast_cancer_model.h5",  # File path to save the model
    save_weights_only=False,  # Save the entire model (architecture + weights)
    save_best_only=False,  # Save model after every epoch
    verbose=1  # Print message when saving
)

# Train the model and save after each epoch
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset,
    callbacks=[checkpoint_callback]  # Add the checkpoint callback
)


Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7418 - loss: 0.5253
Epoch 1: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 268s 3s/step - accuracy: 0.7418 - loss: 0.5253 - val_accuracy: 0.6790 - val_loss: 0.5880
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7593 - loss: 0.5031
Epoch 2: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 269s 3s/step - accuracy: 0.7591 - loss: 0.5032 - val_accuracy: 0.6591 - val_loss: 0.5966
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7453 - loss: 0.5038
Epoch 3: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 268s 3s/step - accuracy: 0.7453 - loss: 0.5038 - val_accuracy: 0.6591 - val_loss: 0.6021
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7625 - loss: 0.4884
Epoch 4: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 267s 3s/step - accuracy: 0.7624 - loss: 0.4885 - val_accuracy: 0.6804 - val_loss: 0.5821
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7727 - loss: 0.4884
Epoch 5: saving model to breast_cancer_model.h5


...

90/90 ━━━━━━━━━━━━━━━━━━━━ 268s 3s/step - accuracy: 0.7597 - loss: 0.4853 - val_accuracy: 0.6307 - val_loss: 0.6278
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7839 - loss: 0.4793
Epoch 7: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 267s 3s/step - accuracy: 0.7840 - loss: 0.4792 - val_accuracy: 0.6861 - val_loss: 0.5825
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7649 - loss: 0.4772
Epoch 8: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 267s 3s/step - accuracy: 0.7650 - loss: 0.4771 - val_accuracy: 0.6889 - val_loss: 0.5772
Epoch 9/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7929 - loss: 0.4590
Epoch 9: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 268s 3s/step - accuracy: 0.7928 - loss: 0.4590 - val_accuracy: 0.6889 - val_loss: 0.5764
Epoch 10/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7782 - loss: 0.4679
Epoch 10: saving model to breast_cancer_model.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 272s 3s/step - accuracy: 0.7783 - loss: 0.4678 - val_accuracy: 0.6776 - val_loss: 0.5847


In [1]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("breast_cancer_model.h5")

# Check the model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 380, 380, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling (Rescaling)         │ (None, 380, 380, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization (Normalization) │ (None, 380, 380, 3)       │               7 │ rescaling[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_1 (Rescaling)       │ (None, 380, 380, 3)       │               0 │ normalization[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 381, 381, 3)       │               0 │ rescaling_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 190, 190, 48)      │           1,296 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 190, 190, 48)      │             192 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 190, 190, 48)      │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 190, 190, 48)      │             432 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 190, 190, 48)      │             192 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 190, 190, 48)      │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 48)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 48)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 12)          │             588 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 48)          │             624 │ block1a_se_reduce[0][0]    │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 18,133,090 (69.17 MB)

 Trainable params: 459,265 (1.75 MB)

 Non-trainable params: 17,673,823 (67.42 MB)

 Optimizer params: 2 (12.00 B)

In [40]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


22/22 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.6989 - loss: 0.5644
Test Loss: 0.5847, Test Accuracy: 0.6776
